In [47]:
# Camera Calibration Using OpenCV
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import time, random
from image_pro import *

def undistort_image(image, camera_matrix, distortion_coeffs, fill_color=(255, 255, 255)):
    h, w = image.shape[:2]
    # Compute the new optimal camera matrix
    new_camera_matrix, valid_pix_roi = cv2.getOptimalNewCameraMatrix(camera_matrix, distortion_coeffs, (w, h), 1, (w, h))

    # Undistort the image
    undistorted_image = cv2.undistort(image, camera_matrix, distortion_coeffs, None, new_camera_matrix)

    # Create a mask of the valid region
    x, y, w, h = valid_pix_roi
    mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
    mask[y:y+h, x:x+w] = 255

    # Invert the mask to get the non-valid regions
    inv_mask = cv2.bitwise_not(mask)

    # Fill the non-valid regions with the fill color
    filled_image = undistorted_image.copy()
    filled_image[inv_mask == 255] = fill_color

    return filled_image

square_size = 29  # in millimeters
board_width = 7
board_height = 5
board_size = (board_width, board_height)

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
# 4x3 grid of points
objp = np.zeros((board_width * board_height, 3), np.float32)
objp[:, :2] = np.mgrid[0:board_width, 0:board_height].T.reshape(-1, 2)

objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

images = glob.glob('*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, board_size, None)
    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)
        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)


In [56]:
# Initialize camera
camera = cv2.VideoCapture(2)  # Replace '0' with your camera source

frame_processing_interval = 0  # Process a frame every 4 seconds
last_processed_time = time.time()

while True:
    ret, frame = camera.read()
    frame = undistort_image(frame, mtx, dist)
    # print(f"width = {frame.shape[1]}, height = {frame.shape[0]}")
    if not ret:
        break

    current_time = time.time()
    if current_time - last_processed_time >= frame_processing_interval:
        last_processed_time = current_time

        # Process the frame
        image_processor = ImageProcessor(frame)
        object_centers = image_processor.detect_objects()

        # Visualize the processed frame
        processed_frame = image_processor.draw_rectangles()
        cv2.imshow("Processed Frame", processed_frame)

        # Select three random objects
        if len(object_centers) > 3:
            selected_objects = random.sample(object_centers, 3)
        else:
            selected_objects = object_centers

        # Proceed with the selected objects for path planning
        # ... (rest of the path planning logic)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
camera.release()
cv2.destroyAllWindows()

In [86]:
def image2Dworld3D(im_P):
    robot_coords = np.array([[-157.84, -132], [-457.84, -132], [-157.84, 95], [-457.84, 95]])  
    image_coords = np.array([[0, 0], [640, 0], [0, 480], [640, 480]])  

    H, _ = cv2.findHomography(image_coords, robot_coords)
    
    point_img = np.array(im_P, dtype=np.float32)
    point_img = np.append(point_img, 1)
    point_world = np.dot(H, point_img)  

    point_world /= point_world[2]
    point_world = point_world[:2]
    
    z_constant = -520
    np.append(point_world, z_constant)
    
    return point_world

In [1]:
from delta_robot import DeltaRobot
r = 138.25
h = 200
s = 50
k = 400
Zt = 0
robot = DeltaRobot(r, h, s, k, Zt)

In [2]:
robot.calculate_fwd_kinematics(20, 20, 0)

array([ -27.29,  -47.27, -324.65])

In [3]:
robot.calculate_inv_kinematics(-27.29, -47.27, -324.65)

[11.84, 11.84, -0.0]

In [4]:
import math

def intersect_circles(x1, y1, r1, x2, y2, r2):
    dist = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

    if dist > r1 + r2 or dist < abs(r1 - r2) or dist == 0:
        return None  

    a = (r1**2 - r2**2 + dist**2) / (2 * dist)
    h = math.sqrt(r1**2 - a**2)

    x3 = x1 + a * (x2 - x1) / dist
    y3 = y1 + a * (y2 - y1) / dist

    intersection1 = (x3 + h * (y2 - y1) / dist, y3 - h * (x2 - x1) / dist)
    intersection2 = (x3 - h * (y2 - y1) / dist, y3 + h * (x2 - x1) / dist)

    return intersection1 if intersection1[0] > intersection2[0] else intersection2

# Example usage:
print(intersect_circles(0, 0, 5, 10, 0, 5))


(5.0, 0.0)


In [1]:
image = cv2.imread("./tomato_images/T7.jpg")
image_processor = ImageProcessor(image)
image_processor.detect_objects()
processed_image = image_processor.draw_rectangles()
desired_width = 300  # or any width that fits your screen
desired_height = 400 # or any height that fits your screen
resized_image = cv2.resize(processed_image, (desired_width, desired_height))

# Display the resized image
cv2.imshow("Processed Image", resized_image)
cv2.waitKey(0)

NameError: name 'cv2' is not defined

In [8]:
image.shape

(4000, 3000, 3)

In [11]:
from image_pro import *
import cv2
import cvlib as cvl
import numpy as np
import matplotlib.pyplot as plt

image = cv2.imread("./tomato_images/T7.jpg")
# image_processor = ImageProcessor(image)
# image_processor.detect_objects()
# processed_image = image_processor.draw_rectangles()
processed_image = cvl.detect_common_objects(image)
# desired_width = 300  # or any width that fits your screen
# desired_height = 400 # or any height that fits your screen
# resized_image = cv2.resize(processed_image, (desired_width, desired_height))

print(processed_image)

([[852, 955, 1899, 2069], [1757, 1459, 2768, 2468]], ['orange', 'orange'], [0.9216803908348083, 0.673895001411438])


In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')


cap = cv2.VideoCapture(0)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

In [ ]:
from pylabel import importer

category_mapping = {
    'b_fully_ripened': 'fully_ripened',
    'l_fully_ripened': 'fully_ripened',
    'b_half_ripened': 'half_ripened',
    'l_half_ripened': 'half_ripened',
    'b_green': 'green',
    'l_green': 'green',
}

path_to_coco_json = "d:/DELL XPS Backup Bishal/Bishal/DELTA/Datasets/laboro/annotations/test.json"
dataset = importer.ImportCoco(path_to_coco_json, name="test_annots")

for annotation in dataset.df.iterrows():
    annotation = annotation[1]  
    new_cat = category_mapping.get(annotation["cat_name"])
    if new_cat:
        dataset.df.at[annotation.name, "cat_name"] = new_cat

dataset.df['cat_name'] = dataset.df['cat_name'].astype('category')
dataset.df['cat_id'] = dataset.df['cat_name'].cat.codes


dataset.export.ExportToYoloV5(output_path="d:/DELL XPS Backup Bishal/Bishal/DELTA/Datasets/laboro/val/labels/")


In [ ]:
import os
import shutil

def move_xml_files(source_dir, target_dir):

    # Ensure target directory exists, if not, create it
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Count of files moved
    files_moved = 0

    # Iterate over all files in the source directory
    for filename in os.listdir(source_dir):
        if filename.endswith('.xml'):
            # Construct full file path
            source_file = os.path.join(source_dir, filename)
            target_file = os.path.join(target_dir, filename)

            # Move the file
            shutil.move(source_file, target_file)
            files_moved += 1
            print(f"Moved: {source_file} to {target_file}")

    print(f"Total of {files_moved} .xml files moved from {source_dir} to {target_dir}")

# Usage example:
images_dir = "d:/DELL XPS Backup Bishal/Bishal/DELTA/Datasets/Drive/Tomato Images/Images"
annots_dir = "d:/DELL XPS Backup Bishal/Bishal/DELTA/Datasets/Drive/Tomato Images/Annotations"
move_xml_files(images_dir, annots_dir)


In [ ]:
from pylabel import importer

label_path = "d:/DELL XPS Backup Bishal/Bishal/DELTA/Datasets/Drive/Tomato Images/Annotations"
path_to_yolo = "d:/DELL XPS Backup Bishal/Bishal/DELTA/Datasets/Drive/Tomato Images/Labels"
dataset_name = "drive images"

dataset = importer.ImportVOC(label_path, dataset_name)
dataset.export.ExportToYoloV5(path_to_yolo)

In [ ]:
import os
import shutil
import random
import logging

logging.basicConfig(level=logging.INFO)

def list_files_in_directory(directory):
    logging.info(f"Listing files in {directory}:")
    if os.path.exists(directory):
        files = os.listdir(directory)
        for file in files:
            logging.info(file)
    else:
        logging.error(f"Directory does not exist: {directory}")

def move_files(files, source, destination):
    for file in files:
        source_file = os.path.join(source, file)
        destination_file = os.path.join(destination, file)

        if os.path.exists(source_file) and os.path.getsize(source_file) > 0:
            shutil.move(source_file, destination_file)
            logging.info(f"Moved: {source_file} to {destination_file}")
        else:
            logging.warning(f"File not found or is empty: {source_file}")

def split_data_set(image_dir, label_dir, train_size=0.8):
    list_files_in_directory(image_dir)  # List files for diagnostics

    images = [file for file in os.listdir(image_dir) if file.endswith(('.jpg', '.jpeg', '.png'))]
    if not images:
        logging.error("No image files found. Check the directory and file extensions.")
        return

    random.shuffle(images)
    split_point = int(len(images) * train_size)

    train_images = images[:split_point]
    val_images = images[split_point:]

    for folder in ['train', 'val']:
        for subdir in [image_dir, label_dir]:
            os.makedirs(os.path.join(subdir, folder), exist_ok=True)

    move_files(train_images, image_dir, os.path.join(image_dir, 'train'))
    move_files(val_images, image_dir, os.path.join(image_dir, 'val'))
    move_files([f.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt') for f in train_images], label_dir, os.path.join(label_dir, 'train'))
    move_files([f.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt') for f in val_images], label_dir, os.path.join(label_dir, 'val'))

    logging.info(f"Dataset split into {len(train_images)} training and {len(val_images)} validation samples.")

# Usage
path_to_yolo = "d:/DELL XPS Backup Bishal/Bishal/DELTA/Datasets/Drive/Tomato Images/Labels"
path_to_images = "d:/DELL XPS Backup Bishal/Bishal/DELTA/Datasets/Drive/Tomato Images/Images"

split_data_set(path_to_images, path_to_yolo, train_size=0.8)

In [10]:
import numpy as np
a = np.random.randn(3, 3)
b = np.random.randn(3, 1)

In [11]:
a

array([[ 0.27239664,  0.04891296,  0.25507883],
       [ 1.01454214, -0.33875551, -0.05687084],
       [-0.97649445,  3.10000794, -0.27028196]])

In [12]:
b

array([[ 1.06182036],
       [-0.0777233 ],
       [ 0.84677257]])

In [13]:
a*b

array([[ 0.2892363 ,  0.05193677,  0.2708479 ],
       [-0.07885356,  0.0263292 ,  0.00442019],
       [-0.82686871,  2.62500169, -0.22886735]])